In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from theano.printing import pydotprint
import pickle


## Linear Regression - Housing Data Set

In [23]:
import mxnet as mx
import mxnet.symbol as S
import mxnet.ndarray as N

In [22]:
with open('data/housing.dframe', 'rb') as dframe:
    housing = pickle.load(dframe)
x_train = np.stack([np.ones((len(housing['TAX']))), housing['RM']],axis=1)
y_train = np.array(housing['MEDV'])

# plt.scatter(x_train[:,1], y_train ,marker='x')
# housing.head()

In [6]:
# model setup
x = S.Variable('x')
y = S.Variable('y')
m = S.Variable('m')
theta = S.Variable('theta')
h = S.dot(x, theta)
J = S.sum(S.pow(h - y, 2)) / (2 * m)
# gJ = T.grad(J, theta)

input_args = {}
input_args['x'] = mx.nd.array(x_train, ctx=mx.cpu())
input_args['y'] = mx.nd.array(np.transpose(np.array(y_train,ndmin=2)), ctx=mx.cpu()),
input_args['m'] = mx.nd.array([len(x_train)], ctx=mx.cpu()),
input_args['theta']=  mx.nd.array([[0], [0]], ctx=mx.cpu())

grad_arguments = { 'theta' : mx.nd.ones((2, ), ctx=mx.cpu())}
c_exec = J.bind(ctx=mx.cpu(), 
                args=input_args, 
                args_grad=grad_arguments)

TypeError: Only Accept list of NDArrays or dict of str to NDArray

In [ ]:
out_grad = mx.nd.ones((2,), ctx=mx.cpu())

c_exec.forward()
c_exec.backward(out_grad)
c_exec.outputs[0].asnumpy()
c_exec.grad_arrays

In [ ]:
# TODO: Plotting

## Binary Classification - Pima Indians Diabetes Data Set

In [7]:
with open('data/pima-indians-diabetes.tuple', 'rb') as xtuple:
    x_train, y_train, x_test, y_test = pickle.load(xtuple)
y_train = np.atleast_2d(y_train).transpose()

## Image Classification - MNIST dataset

In [16]:
data_dir = 'data/'
data_shape = (784, )
batch_size = 128

In [11]:

train           = mx.io.MNISTIter(
    image       = data_dir + "train-images-idx3-ubyte",
    label       = data_dir + "train-labels-idx1-ubyte",
    input_shape = data_shape,
    batch_size  = batch_size,
    shuffle     = True,
    flat        = True,
#     num_parts   = kv.num_workers,
#     part_index  = kv.rank
)

val = mx.io.MNISTIter(
    image       = data_dir + "t10k-images-idx3-ubyte",
    label       = data_dir + "t10k-labels-idx1-ubyte",
    input_shape = data_shape,
    batch_size  = batch_size,
    flat        = True,
#     num_parts   = kv.num_workers,
#     part_index  = kv.rank
)


In [19]:
data = mx.symbol.Variable('data')
fc1 = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=128)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2 = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 64)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
fc3 = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=10)
mlp = mx.symbol.Softmax(data = fc3, name = 'mlp')

model = mx.model.FeedForward(
        ctx                = mx.cpu(),
        symbol             = mlp,
        num_epoch          = 10,
        learning_rate      = 0.1,
        momentum           = 0.9,
        wd                 = 0.00001,
        initializer        = mx.init.Xavier(factor_type="in", magnitude=2.34),
#         **model_args
)
batch_end_callback = [mx.callback.Speedometer(batch_size, 50)]
eval_metrics = ['accuracy']


In [21]:
model.fit(
    X                  = train,
    eval_data          = val,
    eval_metric        = eval_metrics,
    kvstore            = None,
    batch_end_callback = batch_end_callback,
    epoch_end_callback = None)

MXNetError: [02:14:07] src/symbol/symbol.cc:155: Symbol.InferShapeKeyword argument name softmax_label not found.
Candidate arguments:
	[0]data
	[1]fc1_weight
	[2]fc1_bias
	[3]fc2_weight
	[4]fc2_bias
	[5]fc3_weight
	[6]fc3_bias
	[7]mlp_label


## Image Classification - CIFAR-10 dataset